# Basic Query with Expression Filter Example

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

In [5]:
%sh aql -c "set key_send true"

In [6]:
%sh aql -f "../aqlScripts/insert.aql"

In [7]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
for(int i=0; i<10; i++){
    Key key = new Key("test", "testset", "key"+i);
    System.out.println("key"+i+ " : "+client.get(null, key));
}

Initialized the client and connected to the cluster.
key0 : (gen:1),(exp:453788607),(bins:(name:Sandra),(age:34))
key1 : (gen:1),(exp:453788607),(bins:(name:Jack),(age:26))
key2 : (gen:1),(exp:453788607),(bins:(name:Jill),(age:20))
key3 : (gen:1),(exp:453788607),(bins:(name:James),(age:38))
key4 : (gen:1),(exp:453788607),(bins:(name:Jim),(age:46))
key5 : (gen:1),(exp:453788607),(bins:(name:Julia),(age:62))
key6 : (gen:1),(exp:453788607),(bins:(name:Sally),(age:32))
key7 : (gen:1),(exp:453788607),(bins:(name:Sean),(age:24))
key8 : (gen:1),(exp:453788607),(bins:(name:Sam),(age:12))
key9 : (gen:1),(exp:453788607),(bins:(name:Susan),(age:42))


In [8]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [9]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,30));
QueryPolicy qp = new QueryPolicy();
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
}



(gen:1),(exp:453788607),(bins:(name:Sean),(age:24))
(gen:1),(exp:453788607),(bins:(name:Jack),(age:26))
(gen:1),(exp:453788607),(bins:(name:Jill),(age:20))


# Modify query to further select on _name_
Find all records where age between 20 and 30  AND  name starts with S

In [ ]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

// Expression Filter imports
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.query.RegexFlag;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,30));

Expression nameExp = Exp.build( 
     Exp.regexCompare("^S.*", RegexFlag.ICASE|RegexFlag.NEWLINE, Exp.stringBin("name")) 
     );

QueryPolicy qp = new QueryPolicy();

qp.filterExp = nameExp;

RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
}



# Write a query to select on _name_  OR _age_
Find all records where age between 20 and 30  OR  name starts with S

This can no longer utilize Secondary Index - will be a full scan with filter expression.

In [ ]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

// Expression Filter imports
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.query.RegexFlag;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(null);

Expression nameExp = Exp.build( 
    Exp.or(
       Exp.regexCompare("^S.*", RegexFlag.ICASE|RegexFlag.NEWLINE, Exp.stringBin("name")),
       Exp.and( 
           Exp.gt(Exp.intBin("age"), Exp.val(20)), 
           Exp.lt(Exp.intBin("age"), Exp.val(30))) 
          )
     );

QueryPolicy qp = new QueryPolicy();
qp.filterExp = nameExp;
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(thisKey);
    System.out.println(r);
}



# Cleanup

In [ ]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [ ]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"